In [1]:
experiment_name = "2024.05.21_01-Llama-3_fine_tuning---EVALUATION"

In [2]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct:de:WfRsH4J"

In [3]:
!pip install -q openai --progress-bar off

In [4]:
!pip install -q langchain --progress-bar off

In [11]:
import openai

In [5]:
import pandas as pd

import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

import pickle

import glob

from typing import Union
from IPython.display import display, HTML

from sklearn.metrics import classification_report

In [6]:
import os
from IPython.display import clear_output

os.environ["OPENAI_API_BASE"] = 'https://api.endpoints.anyscale.com/v1'
os.environ["OPENAI_API_KEY"] = input("INSERT YOUR ANYSCALE API KEY:")

clear_output()

In [7]:
import os
import requests

s = requests.Session()

api_base = os.getenv("OPENAI_API_BASE")
token = os.getenv("OPENAI_API_KEY")
url = f"{api_base}/chat/completions"

In [12]:
client = openai.OpenAI(
    base_url = "https://api.endpoints.anyscale.com/v1",
    api_key = os.environ["OPENAI_API_KEY"])

In [13]:
finetuning_job_id = "eftjob_qw2ksm3qp955yi4f15zqku9irz"

In [14]:
status = client.fine_tuning.jobs.retrieve(finetuning_job_id)

In [17]:
status

FineTuningJob(id='eftjob_qw2ksm3qp955yi4f15zqku9irz', created_at='2024-05-18T17:05:08.952650+00:00', error=None, fine_tuned_model='meta-llama/Meta-Llama-3-8B-Instruct:de:WfRsH4J', finished_at='2024-05-18T17:20:59.752729+00:00', hyperparameters=Hyperparameters(n_epochs=None, context_length=None), model='meta-llama/Meta-Llama-3-8B-Instruct', object=None, organization_id=None, result_files=['file_vgfnc31k7gvhw2aijumkj3ehfi'], seed=None, status='succeeded', trained_tokens=9043977, training_file='file_4s62zkzctgk5rr93f44w534gat', validation_file='file_farztyvwzdr1v7jdmghd5b8wtt', estimated_finish=None, integrations=None, creator_id='usr_ff6g3tcylk3suc2ft12w3vw6hs')

In [19]:
file_id = status.result_files[0]
content = client.files.retrieve_content(file_id)

/tmp/ipykernel_3585/2571224947.py:2: DeprecationWarning: The `.content()` method should be used instead
  content = client.files.retrieve_content(file_id)


In [20]:
content

'{"epoch": 0, "iteration": 1, "train_loss": 1.4674596786499023, "trained_tokens": 83440, "valid_loss": null, "perplexity": null, "time_since_job_start": 197.77007579803467}\n{"epoch": 0, "iteration": 2, "train_loss": 1.4756062030792236, "trained_tokens": 165296, "valid_loss": null, "perplexity": null, "time_since_job_start": 205.69420385360718}\n{"epoch": 0, "iteration": 3, "train_loss": 1.4471592903137207, "trained_tokens": 245472, "valid_loss": null, "perplexity": null, "time_since_job_start": 209.85702109336853}\n{"epoch": 0, "iteration": 4, "train_loss": 1.4328726530075073, "trained_tokens": 327936, "valid_loss": null, "perplexity": null, "time_since_job_start": 214.38105130195618}\n{"epoch": 0, "iteration": 5, "train_loss": 1.4073245525360107, "trained_tokens": 409104, "valid_loss": null, "perplexity": null, "time_since_job_start": 217.93808889389038}\n{"epoch": 0, "iteration": 6, "train_loss": 1.3729019165039062, "trained_tokens": 489856, "valid_loss": null, "perplexity": null, "

In [26]:
data = content

# Rimuovi gli apici esterni
data = data.strip()[1:-1]

# Dividi la stringa in base ai caratteri di nuova linea
json_strings = data.split('\\n')

In [27]:
json_strings

['"epoch": 0, "iteration": 1, "train_loss": 1.4674596786499023, "trained_tokens": 83440, "valid_loss": null, "perplexity": null, "time_since_job_start": 197.77007579803467}\n{"epoch": 0, "iteration": 2, "train_loss": 1.4756062030792236, "trained_tokens": 165296, "valid_loss": null, "perplexity": null, "time_since_job_start": 205.69420385360718}\n{"epoch": 0, "iteration": 3, "train_loss": 1.4471592903137207, "trained_tokens": 245472, "valid_loss": null, "perplexity": null, "time_since_job_start": 209.85702109336853}\n{"epoch": 0, "iteration": 4, "train_loss": 1.4328726530075073, "trained_tokens": 327936, "valid_loss": null, "perplexity": null, "time_since_job_start": 214.38105130195618}\n{"epoch": 0, "iteration": 5, "train_loss": 1.4073245525360107, "trained_tokens": 409104, "valid_loss": null, "perplexity": null, "time_since_job_start": 217.93808889389038}\n{"epoch": 0, "iteration": 6, "train_loss": 1.3729019165039062, "trained_tokens": 489856, "valid_loss": null, "perplexity": null, "

In [28]:
# Converti ciascuna stringa JSON in un dizionario Python
list_of_dicts = [json.loads(json_str) for json_str in json_strings]

JSONDecodeError: Extra data: line 1 column 8 (char 7)

In [ ]:
# Stampa il risultato
for d in list_of_dicts:
    print(d)

In [29]:
# Rimuovi gli apici esterni
data = data.strip()[1:-1]

# Sostituisci "null" con "None" per renderlo compatibile con il parsing JSON di Python
data = data.replace("null", "None")

# Dividi la stringa in base ai caratteri di nuova linea
json_strings = data.split('\\n')

# Converti ciascuna stringa JSON in un dizionario Python
list_of_dicts = [eval(json_str) for json_str in json_strings]

# Stampa il risultato
for d in list_of_dicts:
    print(d)

SyntaxError: unterminated string literal (detected at line 1) (<string>, line 1)

In [30]:
# Rimuovi gli apici esterni
data = data.strip()[1:-1]

# Sostituisci "null" con "None"
data = data.replace("null", "None")

# Ripristina gli apici singoli attorno ai valori di testo
json_strings = data.split('\\n')
json_strings = [s.replace('"None"', 'null') for s in json_strings]  # Revert to valid JSON

# Converti ciascuna stringa JSON in un dizionario Python
list_of_dicts = [json.loads(s) for s in json_strings]

# Stampa il risultato
for d in list_of_dicts:
    print(d)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)